In [1]:
#import all the required libraries

#make sure all these libraries are installed 

!conda install -c conda-forge selenium --yes # Install selenium for conda
!conda install -c anaconda beautifulsoup4 --yes #Install beautifulsoup
!conda install -c anaconda lxml --yes
!conda install -c anaconda html5lib --yes
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c anaconda wget --yes


import numpy as np
import urllib.request
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import folium # map rendering library
print("All packages have been imported")
#There are Multiple parts to this assignment..
#1st is web scraping

Solving environment: - 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2019.11.28-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::certifi-2019.11.28-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::certifi-2019.11.28-py37_0
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0
  - anaconda/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::cer

In [2]:
# URL for the wikipedia page is given below
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# assign a variabl to the page or import the page into a variable
page = urllib.request.urlopen(wiki_url)

In [3]:
#convert the page (HTML) into beautiful soupe format
soup = BeautifulSoup(page, "html5lib")
# soup
# print(soup.prettify())

In [4]:
#class="wikitable sortable jquery-tablesorter"
all_tables=soup.find_all("table")
# all_tables
right_table=soup.find('table', class_="wikitable sortable")
# right_table


In [5]:
# assigning variables which will be used for loading values in dataframe which will be created later
a,b,c=[],[],[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        a.append(cells[0].find(text=True))
        b.append(cells[1].find(text=True))
        c.append(cells[2].find(text=True))

        

# ##uncomment this in order to add a empty neigbourhood with some boruough for validaitng point 4
# a.append('V12B')
# b.append('test')
# c.append('\n')

In [6]:
df=pd.DataFrame(a,columns=['Postal Code'])
df['Borough']=b
df['Neighbourhood']=c
# Replacing \n artifact that we got during creating table from soup
df = df.replace('\n',' ', regex=True)

In [7]:
df_test = df
# removing the empty rows
# df_test=df_test[df_test['Neighbourhood'].str.strip().astype(bool)] #

df_test = df_test[~df_test.Borough.str.contains("Not assigned")]

# checking to see if there are any couloums that are empty
# Empty_coloumn_test = df_test[df_test['Borough'].str.contains("Not assigned")] #Not assigned

#Reseting the index in order to start indexing from 0
# df_test.reset_index(inplace=True)
# Removing the extra index column that we got after using reset_index object
# df_test.drop(columns=['index'],inplace=True)
# appending all the strings together which we get by groupby
df_test = df_test.groupby(['Postal Code', 'Borough'], as_index=False).agg({'Neighbourhood': ' '.join})

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

# replace the empty Neighbourhood with borough
df_test['Neighbourhood'] = np.where((df_test['Neighbourhood'].str.strip().astype(bool) == False), df_test['Borough'], df_test['Neighbourhood'])


df_test


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [8]:
df_test.shape
##comments lines where used for testing
# df_test1 = df_test
# df_test1[df_test1['Postal Code'].str.contains("M9V")]
# df_test[df_test['Neighbourhood'].str.strip().astype(bool)]

(103, 3)

In [9]:
##geo encoding not working

## Using Geo encoder package
# !conda install -c conda-forge geocoder --yes

# import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(df_test['Postal Code']))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [10]:
## getting the geo encoding data using csv file
!wget -q -O 'test.csv' http://cocl.us/Geospatial_data

Solving environment: done

# All requested packages already installed.



In [11]:
customers_df = pd.read_csv('test.csv')
customers_df
# df_test.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
# df['Postal Code']
df_test['Postal Code'] = df_test['Postal Code'].astype('string') 
customers_df['Postal Code'] = customers_df['Postal Code'].astype('string')
##merge isn't working for some reason
# MI = pd.merge(df_test , customers_df, left_on='Postal Code',right_on='Postal Code')
MI = pd.concat([df_test,customers_df], axis=1, join='inner')
df3=MI
LANGLOT = df3.loc[:, ~df3.columns.duplicated()]
LANGLOT

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
